In [3]:
!pip install chembl_webresource_client

In [3]:
import pandas as pd

In [25]:
### Task 1 ###

from chembl_webresource_client.new_client import new_client
drug = new_client.drug
approved_drugs = drug.filter(max_phase=4).order_by('first_approval')
print("Number of approved drugs:", len(approved_drugs))
approved_drugs[0][]

Number of approved drugs: 14805


In [51]:
ChemblID = [x['molecule_chembl_id'] for x in approved_drugs]
names = [next((synonym['molecule_synonym'] for synonym in x['molecule_synonyms'] if synonym['syn_type'] == 'FDA'), 'N/A') for x in approved_drugs]
years = [x['first_approval'] for x in approved_drugs]

df = pd.DataFrame({'Chembl_ID': ChemblID, 'Name': names, 'First approval year': years})
df_sorted = df.sort_values(by=['First approval year', 'Name'])
df_sorted.reset_index(drop=True, inplace=True)
print(df_sorted)

           Chembl_ID                          Name  First approval year
0          CHEMBL449                  Butabarbital               1939.0
1      CHEMBL1200542  Desoxycorticosterone acetate               1939.0
2          CHEMBL821                     Guanidine               1939.0
3      CHEMBL1201657                Heparin sodium               1939.0
4           CHEMBL90                     Histamine               1939.0
...              ...                           ...                  ...
14800      CHEMBL986                     Vitamin a                  NaN
14801      CHEMBL986                     Vitamin a                  NaN
14802      CHEMBL986                     Vitamin a                  NaN
14803     CHEMBL1675           Vitamin a palmitate                  NaN
14804   CHEMBL502135                        Xylose                  NaN

[14805 rows x 3 columns]


In [54]:
### Task 2 ###
df_14 = df[df['First approval year'] >= 2014]
df_14

,Chembl_ID,Name,First approval year
1786,CHEMBL139367,N/A,2014.0
1787,CHEMBL125,Miltefosine,2014.0
1788,CHEMBL206253,Netupitant,2014.0
1789,CHEMBL443052,Tavaborole,2014.0
1790,CHEMBL373742,Vasopressin,2014.0
...,...,...,...
2205,CHEMBL4297590,N/A,2022.0
2206,CHEMBL259571,Tapinarof,2022.0
2207,CHEMBL4297634,Gadopiclenol,2022.0
2208,CHEMBL4297666,Omidenepag isopropyl,2022.0


In [55]:
import requests
